In [2]:
from sympy import *
import numpy as np
from sympy.tensor.array import derive_by_array
import torch

## What if I don't want to do things completely by hand?

In [3]:
init_printing()
x, y = symbols("x,y")
def my_funct(var1, var2):
    return var1 * var2 + sin(pi * var1)
my_funct(x,y)

In [4]:
diff(my_funct(x,y), x)

In [5]:
diff(my_funct(x,y), y)

## Evaluate these functions numerically

In [6]:
numeric_diff_x = lambdify((x,y), diff(my_funct(x,y), x), 'numpy')
numeric_diff_y = lambdify((x,y), diff(my_funct(x,y), y), 'numpy')

In [7]:
numeric_diff_x(2,3)

In [8]:
numeric_diff_y(2,3)

## Autograd

In [9]:
x_1 = torch.tensor(2.0, requires_grad=True)
y_1 = torch.tensor(3.0, requires_grad=True)
out = x_1 * y_1 + torch.sin(np.pi * x_1)
out.backward()

In [10]:
x_1.grad

tensor(6.1416)

In [11]:
y_1.grad

tensor(2.)

## Example from Assignment 3

In [12]:
start_dim = np.random.randint(low=2, high=4+1)
i = np.random.rand(start_dim)
j = np.random.rand(start_dim)
k = np.random.rand(start_dim)

In [13]:
def solution_manual_autograd(a, b, y):
    agrad = 2*(2*a**2+a*b+b-y)*(4*a+b)
    bgrad = 2*(2*a**2+a*b+b-y)*(a+1)
    ygrad = 2*(2*a**2+a*b+b-y)*(-1)
    return agrad, bgrad, ygrad

In [14]:
def solution_torch_autograd(a, b, y):
    a = torch.tensor(a, requires_grad=True)
    b = torch.tensor(b, requires_grad=True)
    y = torch.tensor(y, requires_grad=True)
    l = ((2*a**2+a*b+b-y)**2).sum()
    l.backward()
    return a.grad, b.grad, y.grad

### Symbolically

In [15]:
a, b, y = symbols("a, b, y")

In [16]:
y_hat = 2 * a ** 2 + a * b + b
y_hat

In [17]:
loss = (y_hat - y) ** 2
loss

### Q: What happened to the $\sum$ when taking the derivative?

$L = \sum_{i}^{n} (\hat{y}_{i} - y_{i})^2$

We're taking the *partial* derivative with respect to *one* variable at a time.  
For example, w.r.t $y_1$:  

\begin{align*}
\frac{\partial L}{\partial y_{1}} &= \frac{\partial}{\partial y_{1}} \sum_{i}^{n} (\hat{y}_{i} - y_{i})^2 \\
&= \sum_{i}^{n} \frac{\partial}{\partial y_{1}}(\hat{y}_{i} - y_{i})^2 \\
&= \frac{\partial}{\partial y_{1}}(\hat{y}_{1} - y_{1})^2 + \frac{\partial}{\partial y_{1}}(\hat{y}_{2} - y_{2})^2 + \text{ ... } +\frac{\partial}{\partial y_{1}}(\hat{y}_{n} - y_{n})^2 \\
&= \frac{\partial}{\partial y_{1}}(\hat{y}_{1} - y_{1})^2 + 0 + \text{ ... } + 0
\end{align*}

So, for the partial derivative w.r.t a particular variable, although the sum is there, all the other terms are treated as a derivative of a constant ($= 0$) as they concern other variables and not the one we're interested in.


### The derivatives below are w.r.t one $y, a$, and $b$

In [18]:
diff(loss, y)

In [19]:
diff(loss, a)

In [20]:
diff(loss, b)

### Need to evaluate the equations numerically

In [21]:
numeric_ygrad = lambdify((a, b, y), diff(loss, y), 'numpy')
numeric_bgrad = lambdify((a, b, y), diff(loss, b), 'numpy')
numeric_agrad = lambdify((a, b, y), diff(loss, a), 'numpy')

In [22]:
sym_ygrad, sym_bgrad, sym_agrad = numeric_ygrad(i,j,k), numeric_bgrad(i,j,k), numeric_agrad(i,j,k)

In [23]:
# partial derivative w.r.t y_1, where y_1 == i[0]; w.r.t y_2, where y_2 == i[1] etc.
sym_ygrad

array([-2.00403709, -2.5494534 ])

In [24]:
man_agrad, man_bgrad, man_ygrad = solution_manual_autograd(i,j,k)

In [25]:
assert np.allclose(sym_agrad, man_agrad)
assert np.allclose(sym_bgrad, man_bgrad)
assert np.allclose(sym_ygrad, man_ygrad)
print(f"All True")

All True


In [26]:
torch_agrad, torch_bgrad, torch_ygrad = solution_torch_autograd(i,j,k)

In [27]:
assert np.allclose(sym_agrad, torch_agrad)
assert np.allclose(sym_bgrad, torch_bgrad)
assert np.allclose(sym_ygrad, torch_ygrad)
print(f"All True")

All True
